In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
chromedriver = 'D:/20220103_lab/14.python/data/chrome/chromedriver'
driver = webdriver.Chrome(chromedriver) 

# 포스팅 작성 당시 크롬 버젼 : 92

In [ ]:
df = pd.DataFrame

In [ ]:
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''

In [ ]:
# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

In [ ]:
# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [ ]:
# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
Python


In [ ]:
from selenium import webdriver
import time

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

In [ ]:
# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [53]:
# 여러 페이지(999)에서 반복하기
for i in range(999):
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")
    

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

            # 가게 주소 데이터 수집 // span.section-result-location

            addr = s.find_element_by_css_selector("span.section-result-location").text

            print(title, "/", score, "/", addr)
            
    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2762349343.py:6: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  stores = driver.find_elements_by_css_selector("div.section-result-content")
C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2762349343.py:28: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")


데이터 수집 완료.


In [54]:
stores

[]

In [ ]:
# 크롬창 닫기
driver.close()
# 출처: https://datalabbit.tistory.com/45 [간토끼 DataMining Lab:티스토리]

In [18]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [19]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [20]:
driver.get("https://www.google.com/maps/")

In [21]:
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2243747384.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  searchbox = driver.find_element_by_css_selector("input#searchboxinput")


In [22]:
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1190549724.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")


In [126]:
for i in range(999):
    time.sleep(3)
    stores = driver.find_elements_by_css_selector("QA0Szd")
    for s in stores:
        print(s)
        title = s.find_element_by_css_selector("h3.section-result-title").text
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"
            
            addr = s.find_element_by_css_selector("span.section-result-location").text
            
            print(title, "/", score, "/", addr)
            
    try:
        nextpage = driver.find_element_by_css_selebctor("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

데이터 수집 완료.


C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1891611790.py:3: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  stores = driver.find_elements_by_css_selector("QA0Szd")


In [65]:
s

NameError: name 's' is not defined

In [63]:
title

NameError: name 'title' is not defined

In [ ]:
driver.close()

In [33]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [34]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [35]:
driver.get("https://www.google.com/maps/")

In [36]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [37]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [66]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]')

In [39]:
target.text

'평점\n영업시간\n모든 필터\n아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n지금 영업 중 · 032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n이지동물병원\n4.5(8)\n동물 병원 · 병방동 422\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-549-0077\n경로\n신통방통동물병원\n4.9(10)\n동물 병원 · 작전동 863\n영업 중 ⋅ 오후 8:00에 영업 종료 · 032-552-3369\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n작전24동물병원\n4.4(7)\n동물병원 · 작전동 925-1\n경로\n재영동물병원\n3.8(4)\n동물병원 · 계산1동\n지금 영업 중 · 032-542-4647\n경로\n검색결과 1 - 20 표시 중\n지도 이동 시 결과 업데이트\n로그인\n레이어\n지도 데이터 ©2022 TMap Mobility대한민국약관개인정보처리방침\n의견 보내기\n500m'

In [170]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [171]:
driver.execute_script("window.scrollTo(0, 1000)") 

In [43]:
last_height = driver.execute_script("return document.body.scrollHeight")

In [44]:
last_height

887

In [45]:
SCROLL_PAUSE_SEC = 1
while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [50]:
# ActionChains 를 사용하기 위해서.
from selenium.webdriver import ActionChains

# id가 something 인 element 를 찾음
some_tag = driver.find_element(By.ID, 'pane')

# somthing element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(some_tag).perform()


In [8]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

JavascriptException: Message: javascript error: Cannot read properties of null (reading 'scrollTo')
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x009FF154+848212]
	Ordinal0 [0x009FF012+847890]
	Ordinal0 [0x009FF98A+850314]
	Ordinal0 [0x00A550C9+1200329]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [ ]:
for divtag in target.find_elements(By.XPATH, './div'):
    if divtag.text != "":
        print("--------------------------")
        atag = divtag.find_element(By.XPATH, './div/a[1]')
        subdiv = divtag.find_element(By.XPATH, "./div/div[2]")
        name = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
        star = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
        addr = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]")

        print(name.text)
        print(addr.text)
        print(atag.get_attribute('href').split('!')[5][2:])
        print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
        print(star.text)
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

In [144]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_20184/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [4]:
driver.get("https://map.kakao.com/")

In [311]:
search_input_tag = driver.find_element(By.ID, 'search.keyword.query')
search_input_tag.send_keys('동물병원')
time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search.keyword.query"]"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A56372+1205106]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [6]:
search_button_tag = driver.find_element(By.ID, 'search.keyword.submit')
search_button_tag.click()
time.sleep(5)

In [8]:
place_more = driver.find_element(By.ID, 'info.search.place.more')
place_more.click()
time.sleep(5)

In [40]:
for i in range(999):
    nextpage = driver.find_element(By.ID, "info.search.page.no{}".format((i+1)%5))
    nextpage.click()
    print(i)
    if i == 4 :
        nextpage = driver.find_element(By.ID, "info.search.page.next").click()
        time.sleep(1)
        i = 1

0
1
2
3
4


In [223]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [312]:
search_input_tag = driver.find_element(By.CLASS_NAME, 'mL3xi')
search_input_tag.send_keys('동물병원')
time.sleep(3)


In [314]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/451005478.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [315]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)

In [316]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [229]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")
driver.get("https://www.google.com/maps/")
time.sleep(3)
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(3)
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/696269896.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


KeyboardInterrupt: 

In [259]:
driver.execute_script("document.body.style.zoom='100%'")

#elem.send_keys(Keys.ENTER)

In [288]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[2]')

In [258]:
target.text

'아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-9975\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546-0079\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n032-553-4652\n웹사이트\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582\n웹사이트\n경로\n부평종합동물병원\n4.5(101)\n동물병원 · 부평동 부평대로 154\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-511-6836\n경로\n보비동물병원\n4.6(48)\n포근한동물병원\n4.9(31)\n동물병원 · 중동 1168\n24시간 영업 · 032-231-0075\n경로\n펫사랑동물병원\n4.4(48)\n동물병원 · 산곡동 194-6\n032-521-0003\n웹사이트\n경로\n24시나음동물병원\n4.6(45)\n동물 병원 · 중동\n032-323-7581\n경로\n부천 TLC 동물의료센터\n4.9(35)\n동물병원 · 중동 길주로 281 TLC 동물의료센터 1층\n24시간 영업 · 032-323-2475\n웹사이트\n경로\n동행동물병원\n4.7(52)\n동물병원 · 양도로 19번길 15\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 031-986-9816\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n신통방통동물병원\n4.9(10)\n동물 

In [162]:
panee = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[8]')

class = "m6QErb DxyBCb kA9KIf dS8AEf ecceSd"

In [193]:
asdasdf = driver.find_element_by_css_selector("div.rHNip.cRLbXd")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1298259985.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  asdasdf = driver.find_element_by_css_selector("div.rHNip.cRLbXd")


In [191]:
asdasdf.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A2A2D3+1024723]
	Ordinal0 [0x00A20083+983171]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [194]:
asdasdf.send_keys(Keys.END)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A249C3+1001923]
	Ordinal0 [0x00A240A5+999589]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [203]:
akjsvebjkseagvf2 = driver.find_element_by_class_name("rHNip cRLbXd")


C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3682334982.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  akjsvebjkseagvf2 = driver.find_element_by_class_name("rHNip cRLbXd")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".rHNip cRLbXd"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A56372+1205106]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [208]:
akjsvebjkseagvf2 = driver.find_element_by_css_selector("div.drGLxe")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/3934446620.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  akjsvebjkseagvf2 = driver.find_element_by_css_selector("div.drGLxe")


In [212]:
akjsvebjkseagvf2.send_keys(Keys.ENTER)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A249C3+1001923]
	Ordinal0 [0x00A240A5+999589]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [205]:
akjsvebjkseagvf = driver.find_element(By.CLASS_NAME, "tXNTee LCTIRd L6Bbsd VbYL9")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tXNTee LCTIRd L6Bbsd VbYL9"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A56372+1205106]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [195]:
jksdfkhsbdf = driver.find_element_by_css_selector("div.drGLxe")

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/2244094299.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  jksdfkhsbdf = driver.find_element_by_css_selector("div.drGLxe")


In [196]:
jksdfkhsbdf.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC610+837136]
	Ordinal0 [0x00A2A2D3+1024723]
	Ordinal0 [0x00A20083+983171]
	Ordinal0 [0x00A4427C+1131132]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77977A4E+238]


In [328]:
target.text

'세림동물병원\n5.0(23)\n동물병원 · 산곡동 143-2\n032-503-0075\n경로\n길동물병원\n4.0(7)\n동물병원 · KR 인천광역시 서구 신현동 303 금강아미움 길동물병원 2층\n영업 종료 ⋅ 수 오전 10:00에 영업 시작 · 032-578-5119\n웹사이트\n경로\n이오동물병원\n5.0(5)\n동물 병원 · 검암동 506-1번지 동곡프라자 1층 108호\n032-710-7553\n경로\n부천 TLC 동물의료센터\n4.9(35)\n동물병원 · 중동 길주로 281 TLC 동물의료센터 1층\n24시간 영업 · 032-323-2475\n웹사이트\n경로\n포근한동물병원\n4.9(31)\n동물병원 · 중동 1168\n24시간 영업 · 032-231-0075\n경로\n24시나음동물병원\n4.6(45)\n동물 병원 · 중동\n032-323-7581\n경로\n청라종합동물병원\n5.0(13)\n동물병원 · 119(118 KR 인천광역시 서구 청라커낼로 252 롯데캐슬단지내상가 C동\n영업 종료 ⋅ 수 오전 10:00에 영업 시작 · 050-71318-0367\n웹사이트\n경로\n짱구네동물병원\n3.7(6)\n동물병원 · 갈산동 59-5\n032-511-5515\n경로\n청라루비동물병원\n4.3(34)\n동물병원 · 청라동 루비로134번길 6-4\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-568-0922\n웹사이트\n경로\n24시 해든동물메디컬센터\n4.2(81)\n동물병원 · 중동 신흥로 1106 KR 190) 위브더스테이트 1단지상가 2층 213-216호\n24시간 영업 · 032-715-7579\n웹사이트\n경로\n펫사랑동물병원\n4.4(48)\n동물병원 · 산곡동 194-6\n032-521-0003\n웹사이트\n경로\n지중해동물병원\n4.8(34)\n동물 병원 · 중동 1058-2번지 107호 중동프라자 부천시 경기도 KR\n032-321-7603\n경로\n리치동물병원\n4.0(2)\n동물병원 · 삼산동 389

In [187]:
target.send_keys(Keys.END)

In [155]:
driver.execute_script("window.scrollTo(0, 10000)") 

In [159]:
driver.execute_script('arguments[0].scrollIntoView(true);', target)

In [169]:
actions = ActionChains(driver) \
    .move_to_element(panee) \
    .click() \
    .send_keys(Keys.END)

actions.perform() 

In [ ]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

divtag -> class -> "m6QErb DxyBCb kA9KIf dS8AEf ecceSd QjC7t"

subdiv -> jstcache="184" 들 모아놓음

name -> class="NrDZNb"

star -> jstcache="83"

In [295]:
target

<selenium.webdriver.remote.webelement.WebElement (session="53bf810fa272506cc2495a7a82b04fe4", element="4b490803-0c2a-4711-bfb2-de0b7edc2a46")>

In [389]:
target.text

'방문 전에 관련 정보의 확인을 위해 1339콜센터에 연락하시기 바랍니다 자세히 알아보기\n웨스턴동물의료센터\n4.4(16)\n· 노고산동 신촌로 110\n24시간 영업 · 02-701-7580\n웹사이트\n경로\n레이동물병원\n4.7(19)\n동물병원 · 송정로 20 KR 레이동물병원\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 031-245-7707\n웹사이트\n경로\n쿨펫동물병원\n리뷰 없음\n동물병원 · 구월1동 1130\n032-437-7582\n경로\n탑동물병원\n리뷰 없음\n동물 관련업 · 심곡본동 687-4\n032-663-0275\n경로\n비바동물병원\n5.0(1)\n동물병원 · 쑥고개로 57-1\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 02-872-7582\n경로\n녹양 돌봄 동물병원\n5.0(10)\n동물병원 · 녹양동 비우로 100 KR 1층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 031-829-7577\n웹사이트\n경로\n우리동물병원\n리뷰 없음\n동물 병원 · 용현동 622-129\n032-203-0085\n경로\n시민동물병원\n리뷰 없음\n동물 병원 · 주안동 1455-8\n032-868-2646\n경로\n힐링동물병원\n1.0(1)\n동물 병원 · 신공덕동 172번지 펜트라하우스 105동 B235호 마포구 서울특별시 KR\n02-713-8275\n경로\n가톨릭대학교 인천성모병원\n3.7(125)\n대학병원 · 동수로 56\n영업 종료 ⋅ 수 오전 8:30에 영업 시작 · 1544-9004\n웹사이트\n경로\n우리애동물병원\n5.0(1)\n동물 병원 · 논현동 671-1번지 1층 109호 D아파트 PLAZA 남동구 인천광역시 KR\n032-271-2202\n웹사이트\n경로\n경기가축병원\n리뷰 없음\n동물 병원 · 주안동 1417-93\n032-867-2270\n경로\n오아시스 정형외과 신경외과 동물병원\n3.9(27)\n동물병원 · 센트럴타운로 107, 광교 푸르지오 월

In [322]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[2]')

In [323]:
lname = []
lstar = []
laddr = []

In [402]:
for divtag in target.find_elements(By.XPATH, "./div[1]"):
    if divtag.text != "":
        for i in lista:
            print("--------------------------")
            atag = divtag.find_element(By.XPATH, "./div[{}]/div[1]/a[1]".format(i))
            subdiv = divtag.find_element(By.XPATH, "./div[{}]".format(i))
            name = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
            star = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
            addr = subdiv.find_element(By.XPATH, "./div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]")

            print(name.text)
            print(star.text)
            print(addr.text)
            
#            print(atag.get_attribute('href').split('!')[5][2:])
#            print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
            
            lname.append(name.text)
            lstar.append(star.text)
            laddr.append(addr.text)
            
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

--------------------------
빌리브 동물병원
리뷰 없음
동물병원 · 성남대로 449
영업 중 ⋅ 오후 9:00에 영업 종료
--------------------------
맘동물병원
리뷰 없음
동물 병원 · 경기 수원시 영통구 이의동 1332
031-548-1175
--------------------------
반려동물암센터
리뷰 없음
동물병원 · 황새울로342번길 15 은성빌딩
031-709-5500
--------------------------
24시 성심 동물 메디컬 센터
4.4(95)
동물병원 · 봉명동 548-11
24시간 영업 · 042-719-7566
--------------------------
KU건국동물병원
4.4(8)
동물 병원 · 예산읍 산성리 669
041-335-1023
--------------------------
한국동물병원협회
리뷰 없음
협회 또는 조직 · 서초2동 1358-1
02-522-4722
--------------------------
녹십자동물병원
4.3(24)
동물병원 · 부여읍 구아리 141-5
영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
--------------------------
충남대학교 동물병원
4.5(14)
동물병원 · 궁동 220번지
영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704
--------------------------
예산동물병원
4.3(3)
동물병원 · 예산읍 주교리 240-5
041-332-5895
--------------------------


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div[21]/div[1]/a[1]"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B7B8F3+2406643]
	Ordinal0 [0x00B0AF31+1945393]
	Ordinal0 [0x009FC748+837448]
	Ordinal0 [0x00A292E0+1020640]
	Ordinal0 [0x00A2957B+1021307]
	Ordinal0 [0x00A1FAE1+981729]
	Ordinal0 [0x00A442C4+1131204]
	Ordinal0 [0x00A1FA64+981604]
	Ordinal0 [0x00A44494+1131668]
	Ordinal0 [0x00A54682+1197698]
	Ordinal0 [0x00A44096+1130646]
	Ordinal0 [0x00A1E636+976438]
	Ordinal0 [0x00A1F546+980294]
	GetHandleVerifier [0x00DE9612+2498066]
	GetHandleVerifier [0x00DDC920+2445600]
	GetHandleVerifier [0x00C14F2A+579370]
	GetHandleVerifier [0x00C13D36+574774]
	Ordinal0 [0x00B11C0B+1973259]
	Ordinal0 [0x00B16688+1992328]
	Ordinal0 [0x00B16775+1992565]
	Ordinal0 [0x00B1F8D1+2029777]
	BaseThreadInitThunk [0x7619FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77977A7E+286]
	RtlGetAppContainerNamedObjectPath [0x7

In [99]:
lista = []
for x in list(range(50)):
    if x % 2 :
        lista.append(x)

In [102]:
lista

[3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49]

In [391]:
print(len(lname))
lname260 = lname

260


In [395]:
len(lname260)

260

In [396]:
print(len(lstar))
lstar260 = lstar

260


In [397]:
print(len(laddr))
laddr260 = laddr

260


In [403]:
print(len(lname))
lname

289


['아마존동물병원',
 '계양동물메디컬센터',
 '해림동물병원',
 '한솔동물병원',
 '아프리카 동물의료센터 [구 아프리카동물병원]',
 '청라라임동물병원',
 '이지동물병원',
 '신통방통동물병원',
 '다오동물병원',
 '하얀기린동물병원',
 '가정동물병원',
 '작전24동물병원',
 '아이케어동물병원',
 '부평종합동물병원',
 '재영동물병원',
 '보비동물병원',
 '신영재동물병원',
 '검암동물병원',
 '부평종합동물의료센터',
 '부천준동물병원',
 '세림동물병원',
 '길동물병원',
 '이오동물병원',
 '부천 TLC 동물의료센터',
 '포근한동물병원',
 '24시나음동물병원',
 '청라종합동물병원',
 '짱구네동물병원',
 '청라루비동물병원',
 '24시 해든동물메디컬센터',
 '펫사랑동물병원',
 '지중해동물병원',
 '리치동물병원',
 '한사랑동물병원',
 '굿모닝동물병원',
 '소나무동물메디컬센터',
 '고강동물병원',
 '24시 SKY 동물병원',
 '위더스동물병원',
 '펫가든동물병원',
 '웰케어동물메디컬센터',
 '닥터팬더동물병원',
 '청천종합동물병원',
 '참동물병원',
 '현대동물병원',
 '동행동물병원',
 '와우동물병원',
 '그랜드동물병원',
 '삼성동물병원',
 '위즈펫동물병원',
 '중동동물병원',
 '하나동물병원',
 '온유동물병원',
 '동수동물병원',
 '루원하이동물병원',
 '이레동물병원',
 '해오름동물병원',
 '백마동물병원',
 '혜인동물병원',
 '파스텔동물병원',
 '서구동물병원',
 'AVC동물메디컬센터',
 '24시동물병원',
 '쿨펫동물병원',
 '다나동물병원',
 '조재진동물병원',
 '24시동물병원',
 '계양동물병원',
 '하노바동물병원',
 '엘림동물병원',
 '비엔동물전문의료센터',
 '쿨펫동물병원이마트계양점',
 '송정종합동물병원',
 '부평모모동물병원',
 'BE동물병원',
 '튼튼동물병원',
 '주주종합동물병원',
 '행복한동물병원',
 '도그앤캣동물병원',
 '부평프라임동물병원',

In [404]:
print(len(lstar))
lstar

289


['4.6(25)',
 '4.3(50)',
 '4.8(100)',
 '4.3(13)',
 '4.4(89)',
 '4.5(181)',
 '4.5(8)',
 '4.9(10)',
 '4.7(6)',
 '5.0(32)',
 '4.0(43)',
 '4.4(7)',
 '4.4(9)',
 '4.5(101)',
 '3.8(4)',
 '4.6(48)',
 '4.0(2)',
 '3.8(13)',
 '4.4(48)',
 '4.8(27)',
 '5.0(23)',
 '4.0(7)',
 '5.0(5)',
 '4.9(35)',
 '4.9(31)',
 '4.6(45)',
 '5.0(13)',
 '3.7(6)',
 '4.3(34)',
 '4.2(81)',
 '4.4(48)',
 '4.8(34)',
 '4.0(2)',
 '3.8(4)',
 '4.0(5)',
 '5.0(2)',
 '3.9(14)',
 '2.3(3)',
 '4.4(13)',
 '5.0(1)',
 '4.1(34)',
 '리뷰 없음',
 '3.5(2)',
 '4.5(14)',
 '4.4(8)',
 '4.7(52)',
 '3.3(7)',
 '리뷰 없음',
 '4.1(26)',
 '리뷰 없음',
 '4.6(21)',
 '3.4(5)',
 '4.0(19)',
 '4.7(18)',
 '5.0(4)',
 '4.9(46)',
 '4.0(9)',
 '3.8(14)',
 '4.6(38)',
 '4.7(20)',
 '4.1(8)',
 '1.0(5)',
 '2.0(1)',
 '리뷰 없음',
 '4.0(4)',
 '4.3(6)',
 '4.0(1)',
 '2.4(5)',
 '4.6(11)',
 '5.0(2)',
 '4.5(95)',
 '리뷰 없음',
 '5.0(1)',
 '4.9(10)',
 '4.3(61)',
 '5.0(1)',
 '4.8(9)',
 '4.2(9)',
 '4.3(8)',
 '4.3(9)',
 '4.5(12)',
 '5.0(1)',
 '리뷰 없음',
 '리뷰 없음',
 '리뷰 없음',
 '3.0(2)',
 '5.0(1)',
 '4.5(1

In [405]:
print(len(laddr))
laddr

289


['동물병원 · 임학동 67-7\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555-7502 ext. 3',
 '동물병원 · 계산동 942-24번지\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-9975',
 '동물병원 · 작전동 864-39\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546-0079',
 '동물병원 · 계산2동 918-2\n032-553-4652',
 '동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582',
 '동물병원 · 청라동 127-2\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:00에 영업 시작 · 032-563-1900',
 '동물 병원 · 병방동 422\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-549-0077',
 '동물 병원 · 작전동 863\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-552-3369',
 '동물병원 · 계산동 1062\n032-548-0275',
 '동물 병원 · 연희동 796-7번지 청라골드클래스커낼웨이 상가동 103호 서구 인천광역시 KR\n031-564-3575',
 '동물병원 · 가정1동 517-32\n032-575-0833',
 '동물병원 · 작전동 925-1',
 '동물병원 · 청라 에메랄드로102번길 10',
 '동물병원 · 부평동 부평대로 154\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-511-6836',
 '동물병원 · 계산1동\n032-542-4647',
 '동물병원 · 산곡동 411-5\n032-524-6959',
 '동물병원 · 병방동 426\n032-552-0075',
 '동물병원 · 검암동 613-1\n032-563-0775',
 '동물병원 · 부평대로 138\n032-511-6836',
 '동물병원 · 중3동 106

In [427]:
asdfa2 = []
for x in laddr:
    asdfa2.append(x.split("\n"))


In [428]:
asdfa2

[['동물병원 · 임학동 67-7',
  '곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555-7502 ext. 3'],
 ['동물병원 · 계산동 942-24번지', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-9975'],
 ['동물병원 · 작전동 864-39', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546-0079'],
 ['동물병원 · 계산2동 918-2', '032-553-4652'],
 ['동물병원 · 장제로340번길 5 3층', '영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582'],
 ['동물병원 · 청라동 127-2', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:00에 영업 시작 · 032-563-1900'],
 ['동물 병원 · 병방동 422', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-549-0077'],
 ['동물 병원 · 작전동 863', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-552-3369'],
 ['동물병원 · 계산동 1062', '032-548-0275'],
 ['동물 병원 · 연희동 796-7번지 청라골드클래스커낼웨이 상가동 103호 서구 인천광역시 KR', '031-564-3575'],
 ['동물병원 · 가정1동 517-32', '032-575-0833'],
 ['동물병원 · 작전동 925-1'],
 ['동물병원 · 청라 에메랄드로102번길 10'],
 ['동물병원 · 부평동 부평대로 154', '곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-511-6836'],
 ['동물병원 · 계산1동', '032-542-4647'],
 ['동물병원 · 산곡동 411-5', '032-524-6959'],
 ['동물병원 · 병방동 426', '032-552-0075'],
 ['동물병원 · 검암동 613-1

In [442]:
pd.DataFrame(asdfa2)[0]

0             동물병원 · 임학동 67-7
1         동물병원 · 계산동 942-24번지
2           동물병원 · 작전동 864-39
3           동물병원 · 계산2동 918-2
4        동물병원 · 장제로340번길 5 3층
                ...          
284       동물 병원 · 예산읍 산성리 669
285    협회 또는 조직 · 서초2동 1358-1
286      동물병원 · 부여읍 구아리 141-5
287           동물병원 · 궁동 220번지
288      동물병원 · 예산읍 주교리 240-5
Name: 0, Length: 289, dtype: object

In [446]:
df_addr3 = pd.DataFrame(df_addr2)

In [448]:
pd.DataFrame(asdfa2)[1]

0      곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1      곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2      곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3                                           032-553-4652
4                  영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
                             ...                        
284                                         041-335-1023
285                                          02-522-4722
286              영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287              영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704
288                                         041-332-5895
Name: 1, Length: 289, dtype: object

In [499]:
df_openphone = pd.DataFrame(pd.DataFrame(asdfa2)[1])

In [500]:
df_openphone

,1
0,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1,곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3,032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
284,041-335-1023
285,02-522-4722
286,영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287,영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704


In [406]:
import pandas as pd

In [408]:
df_name = pd.DataFrame(lname)

In [409]:
df_star = pd.DataFrame(lstar)

In [410]:
df_addr = pd.DataFrame(laddr)

In [414]:
df_name.columns = ["병원이름"]

In [416]:
df_star.columns = ["리뷰"]

In [417]:
df_addr.columns = ["주소"]

In [419]:
df_name.head(5)

,병원이름
0,아마존동물병원
1,계양동물메디컬센터
2,해림동물병원
3,한솔동물병원
4,아프리카 동물의료센터 [구 아프리카동물병원]


In [421]:
df_star.head(5)

,리뷰
0,4.6(25)
1,4.3(50)
2,4.8(100)
3,4.3(13)
4,4.4(89)


In [420]:
df_addr.head(5)

,주소
0,동물병원 · 임학동 67-7\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00...
1,동물병원 · 계산동 942-24번지\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 9...
2,동물병원 · 작전동 864-39\n곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:...
3,동물병원 · 계산2동 918-2\n032-553-4652
4,동물병원 · 장제로340번길 5 3층\n영업 중 ⋅ 오후 10:00에 영업 종료 ·...


In [436]:
df_addr2.head(5)

0         동물병원 · 임학동 67-7
1     동물병원 · 계산동 942-24번지
2       동물병원 · 작전동 864-39
3       동물병원 · 계산2동 918-2
4    동물병원 · 장제로340번길 5 3층
Name: 0, dtype: object

In [447]:
df_addr3.head(5)

,0
0,동물병원 · 임학동 67-7
1,동물병원 · 계산동 942-24번지
2,동물병원 · 작전동 864-39
3,동물병원 · 계산2동 918-2
4,동물병원 · 장제로340번길 5 3층


In [518]:
df_openphone.columns = ["오픈마감시간, 전화번호"]

In [519]:
df_openphone

,"오픈마감시간, 전화번호"
0,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1,곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3,032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
284,041-335-1023
285,02-522-4722
286,영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287,영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704


In [534]:
df_openphone["오픈마감시간, 전화번호"].isnull().sum()

0

In [530]:
len(df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].isnull()])

14

In [533]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].isnull()] = 0

In [544]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == 0] = " "

In [546]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == " "] = "정보가 없습니다."

In [547]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"] == "정보가 없습니다."]

,"오픈마감시간, 전화번호"
11,정보가 없습니다.
12,정보가 없습니다.
37,정보가 없습니다.
39,정보가 없습니다.
49,정보가 없습니다.
71,정보가 없습니다.
79,정보가 없습니다.
89,정보가 없습니다.
184,정보가 없습니다.
202,정보가 없습니다.


11,12,37,39,49,71,79,89,184,202,230,236,255,278

In [482]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains('ext')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [552]:
df_openphone_ext = df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains("ext")]

In [564]:
len(df_openphone_ext["오픈마감시간, 전화번호"][0])

58

In [571]:
df_openphone_ext["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:51]

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/1080749331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_openphone_ext["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:51]


In [582]:
len(df_openphone_ext["오픈마감시간, 전화번호"][58])

49

In [589]:
df_openphone_ext["오픈마감시간, 전화번호"][58] = df_openphone_ext["오픈마감시간, 전화번호"][58][:39]

C:\Users\82103\AppData\Local\Temp/ipykernel_20284/770376131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_openphone_ext["오픈마감시간, 전화번호"][58] = df_openphone_ext["오픈마감시간, 전화번호"][58][:39]


In [594]:
df_openphone["오픈마감시간, 전화번호"][0] = df_openphone_ext["오픈마감시간, 전화번호"][0][:51]

In [595]:
df_openphone["오픈마감시간, 전화번호"][58] = df_openphone_ext["오픈마감시간, 전화번호"][58][:39]

In [607]:
df_openphone["오픈마감시간, 전화번호"][58]

'영업 종료 ⋅ 수 오전 9:30에 영업 시작 · 02-2645-1475'

In [613]:
df_openphone["오픈마감시간, 전화번호"][0][-12:]

'032-555-7502'

In [611]:
df_openphone["오픈마감시간, 전화번호"][58][-12:]

'02-2645-1475'

In [615]:
df_openphone["오픈마감시간, 전화번호"][277][-12:]

' 02-450-3664'

In [ ]:
df_openphone["오픈마감시간, 전화번호"]

In [618]:
df_openphone.loc[df_openphone["오픈마감시간, 전화번호"].str.contains("")]

,"오픈마감시간, 전화번호"
0,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1,곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3,032-553-4652
4,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...
284,041-335-1023
285,02-522-4722
286,영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287,영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704


In [622]:
df_animalhospital = pd.concat([df_name, df_star, df_addr3, df_openphone], axis=1)

In [623]:
df_animalhospital

,병원이름,리뷰,0,"오픈마감시간, 전화번호"
0,아마존동물병원,4.6(25),동물병원 · 임학동 67-7,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1,계양동물메디컬센터,4.3(50),동물병원 · 계산동 942-24번지,곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2,해림동물병원,4.8(100),동물병원 · 작전동 864-39,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3,한솔동물병원,4.3(13),동물병원 · 계산2동 918-2,032-553-4652
4,아프리카 동물의료센터 [구 아프리카동물병원],4.4(89),동물병원 · 장제로340번길 5 3층,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...,...,...,...
284,KU건국동물병원,4.4(8),동물 병원 · 예산읍 산성리 669,041-335-1023
285,한국동물병원협회,리뷰 없음,협회 또는 조직 · 서초2동 1358-1,02-522-4722
286,녹십자동물병원,4.3(24),동물병원 · 부여읍 구아리 141-5,영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287,충남대학교 동물병원,4.5(14),동물병원 · 궁동 220번지,영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704


In [624]:
df_animalhospital.columns = ["병원이름", "리뷰", "주소", "오픈마감전화번호"]

In [625]:
df_animalhospital

,병원이름,리뷰,주소,오픈마감전화번호
0,아마존동물병원,4.6(25),동물병원 · 임학동 67-7,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-555...
1,계양동물메디컬센터,4.3(50),동물병원 · 계산동 942-24번지,곧 영업 종료: 오후 8:00 ⋅ 수 오전 9:30에 영업 시작 · 032-555-...
2,해림동물병원,4.8(100),동물병원 · 작전동 864-39,곧 영업 종료: 오후 8:00 ⋅ 수 오전 10:00에 영업 시작 · 032-546...
3,한솔동물병원,4.3(13),동물병원 · 계산2동 918-2,032-553-4652
4,아프리카 동물의료센터 [구 아프리카동물병원],4.4(89),동물병원 · 장제로340번길 5 3층,영업 중 ⋅ 오후 10:00에 영업 종료 · 032-508-7582
...,...,...,...,...
284,KU건국동물병원,4.4(8),동물 병원 · 예산읍 산성리 669,041-335-1023
285,한국동물병원협회,리뷰 없음,협회 또는 조직 · 서초2동 1358-1,02-522-4722
286,녹십자동물병원,4.3(24),동물병원 · 부여읍 구아리 141-5,영업 종료 ⋅ 수 오전 9:00에 영업 시작 · 041-832-2496
287,충남대학교 동물병원,4.5(14),동물병원 · 궁동 220번지,영업 종료 ⋅ 수 오후 2:00에 영업 시작 · 042-821-6704
